In [1]:
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'])
session = cluster.connect()

In [2]:
import pandas as pd

In [3]:
# Read CSV file using Pandas
df = pd.read_csv('C:/Users/zohreh/Desktop/newprediction.csv')


In [4]:
print(df)

        Date/Time-s      Lat      Lon    Base  prediction
0             16200  40.7476 -73.9871  B02598           1
1             16200  40.7553 -73.9253  B02617           2
2             16260  40.7108 -73.9679  B02598           1
3             16260  40.7341 -73.9890  B02617           1
4             16260  40.7403 -74.0081  B02598           1
...             ...      ...      ...     ...         ...
165508       742620  40.7234 -73.9809  B02764           1
165509       742620  40.7517 -73.9802  B02682           1
165510       742680  40.7857 -73.9764  B02682           2
165511       742980  40.6512 -74.0142  B02682           1
165512       743040  40.7325 -73.9838  B02682           1

[165513 rows x 5 columns]


In [ ]:
keyspace_name ='eshaghi'
query = f"CREATE KEYSPACE {keyspace_name} WITH replication = {{'class': 'SimpleStrategy', 'replication_factor': '1'}}"
session.execute(query)

In [ ]:
# Create table in Cassandra
table_name = 'finaltablenumerictime'
query = f"CREATE TABLE eshaghi.{table_name} (lat DOUBLE, lon DOUBLE, prediction int,\
datetime int,base TEXT,PRIMARY KEY (prediction,lat,lon,datetime,base))"
session.execute(query)

In [ ]:
# Create table in Cassandra for soal 2
table_name = 'finaltablebase'
query = f"CREATE TABLE eshaghi.{table_name} (lat DOUBLE, lon DOUBLE, prediction int,\
datetime int,base TEXT,PRIMARY KEY (base,lat,lon,datetime,prediction))"
session.execute(query)

In [ ]:
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
from shapely.geometry import Point
from datetime import datetime, timedelta

# Create a connection to the Cassandra cluster running on localhost
cluster = Cluster(['localhost'])
session = cluster.connect()

# Set the keyspace
keyspace = 'eshaghi'
session.set_keyspace(keyspace)

# Calculate the datetime range for the past 10 days
end_datetime = datetime.now()
start_datetime = end_datetime - timedelta(days=10)

# Convert datetime objects to milliseconds timestamp
start_timestamp = int(start_datetime.timestamp() * 1000)
end_timestamp = int(end_datetime.timestamp() * 1000)

# Construct the query with the keyspace and table name
query = "SELECT lat, lon FROM finaltablenumerictime WHERE datetime >= 1728000 ALLOW FILTERING"

# Execute the query
statement = SimpleStatement(query)
result_set = session.execute(statement)

# Store the retrieved results in a list
results = list(result_set)

# Initialize a list to store the points
points = []

# Create Point objects for each latitude and longitude coordinate
for result in results:
    lon, lat = result.lon, result.lat
    point = Point(lon, lat)
    points.append(point)

# Print the points
for point in points:
    print(f"Point: {point}")


In [ ]:
# Insert data into Cassandra table
for row in df.itertuples():
    query = f"INSERT INTO eshaghi.{table_name} (lat, lon, prediction, datetime, base) VALUES (%s, %s, %s, %s, %s)"
    session.execute(query, (row.Lat, row.Lon, row.prediction, row._1 , row.Base))

In [37]:
#soal1
from cassandra.cluster import Cluster

cluster = Cluster(['localhost'])
session = cluster.connect()

query = "SELECT prediction, COUNT(*) FROM eshaghi.finaltablenumerictime GROUP BY prediction;"
result_set = session.execute(query)

for row in result_set:
    prediction = row.prediction
    count = row.count
    print(f"Prediction: {prediction}, Count: {count}")


Prediction: 1, Count: 101504
Prediction: 0, Count: 7602
Prediction: 2, Count: 53985
Prediction: 3, Count: 1796


In [38]:
from cassandra.cluster import Cluster

cluster = Cluster(['localhost'])
session = cluster.connect()

query = "SELECT base, COUNT(*) FROM eshaghi.finaltablebase WHERE datetime > 1728000 AND datetime < 2332800 GROUP BY base ALLOW FILTERING;"
result_set = session.execute(query)

for row in result_set:
    base = row.base
    count = row.count
    print(f"Base: {base}, Count: {count}")


Base: B02598, Count: 9213
Base: B02764, Count: 3280
Base: B02512, Count: 1277
Base: B02682, Count: 7528
Base: B02617, Count: 15335


In [33]:
#soal3
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.metadata import KeyspaceMetadata
cluster = Cluster(['localhost'])
session = cluster.connect()
result=list(session.execute('SELECT prediction,COUNT(*) FROM eshaghi.finaltablenumerictime WHERE datetime>1728000 AND datetime<2332800 GROUP BY prediction ALLOW FILTERING;'))
for i in range(0,len(result)):
    result[i]=list(result[i])
result = sorted(result, key=lambda x: x[1], reverse=True)
print("Sorted Clusters:")
for row in result:
    print(f"Cluster {row[0]} has {row[1]} rows in last week.")

Sorted Clusters:
Cluster 1 has 22817 rows in last week.
Cluster 2 has 11621 rows in last week.
Cluster 0 has 1781 rows in last week.
Cluster 3 has 414 rows in last week.


In [22]:
pip install shapely


     ---------------------------------------- 1.4/1.4 MB 247.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [28]:
#soal4
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
from shapely.geometry import Point
from datetime import datetime, timedelta

# Create a connection to the Cassandra cluster running on localhost
cluster = Cluster(['localhost'])
session = cluster.connect()

# Set the keyspace
keyspace = 'eshaghi'
session.set_keyspace(keyspace)

# Calculate the datetime range for the past 10 days
end_datetime = datetime.now()
start_datetime = end_datetime - timedelta(days=10)

# Convert datetime objects to milliseconds timestamp
start_timestamp = int(start_datetime.timestamp() * 1000)
end_timestamp = int(end_datetime.timestamp() * 1000)

# Set the query to retrieve latitude and longitude from the table with the filter condition
query = "SELECT lat, lon FROM finaltablenumerictime WHERE datetime >= 1728000 ALLOW FILTERING"

# Execute the query
statement = SimpleStatement(query)
result_set = session.execute(statement)

# Initialize a dictionary to store the count of each point
point_counts = {}

# Create Point objects for each latitude and longitude coordinate and count the occurrences
for result in result_set:
    lon, lat = result.lon, result.lat
    point = Point(lon, lat)
    
    if point in point_counts:
        point_counts[point] += 1
    else:
        point_counts[point] = 1

# Filter points with count greater than 1
high_traffic_points = [point for point, count in point_counts.items() if count > 1]

# Print the high traffic points and their counts
for point in high_traffic_points:
    count = point_counts[point]
    print(f"Point: {point}, Count: {count}")



Point: POINT (-73.9662 40.5855), Count: 2
Point: POINT (-73.9496 40.616), Count: 2
Point: POINT (-73.9601 40.6371), Count: 2
Point: POINT (-73.9571 40.6399), Count: 2
Point: POINT (-73.9672 40.64), Count: 2
Point: POINT (-74.0095 40.6437), Count: 2
Point: POINT (-74.0002 40.6472), Count: 2
Point: POINT (-73.9787 40.6483), Count: 2
Point: POINT (-74.0041 40.6512), Count: 2
Point: POINT (-74.0044 40.6545), Count: 2
Point: POINT (-74.0106 40.6553), Count: 2
Point: POINT (-74.0092 40.6563), Count: 2
Point: POINT (-74.0034 40.6565), Count: 2
Point: POINT (-74.0071 40.6578), Count: 2
Point: POINT (-73.9885 40.6593), Count: 2
Point: POINT (-73.9828 40.6595), Count: 2
Point: POINT (-73.9803 40.6603), Count: 2
Point: POINT (-73.9802 40.6608), Count: 2
Point: POINT (-73.9938 40.661), Count: 2
Point: POINT (-73.9857 40.6619), Count: 2
Point: POINT (-73.9844 40.6624), Count: 2
Point: POINT (-73.9358 40.6625), Count: 2
Point: POINT (-73.9456 40.6629), Count: 2
Point: POINT (-73.9918 40.663), Count: